<a href="https://colab.research.google.com/github/robrocha42/Desafio_Titanic_Kaggle/blob/main/Desafio_Titanic_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Titanic - Site Kaggle - Orientado a objetos

In [ ]:
from typing import Tuple
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

class DataPrep:
 def __init__(self, data: pd.DataFrame) -> None:
  "Inicializa a classe DataPrep com a base de dados do Titanic."
  self.data = data

 def tratar_nulos(self) -> None:
  "Faz o tratamento das variáveis nulas, imputando o valor adequado."
  # Imputar mediana das idades por classe e sexo
  self.data['Age'] = self.data.groupby(['Pclass', 'Sex'])['Age'] \
  .apply(lambda x: x.fillna(x.median()))
  # Imputar local de embarque
  self.data['Embarked'] = self.data['Embarked'].fillna('S')
 
 def tratar_variaveis_categoricas(self) -> None:
  "Faz o tratamento das variáveis categóricas"
  # Label Encoding da variável Sex
  sexo = {'male': 0, 'female': 1}
  self.data['Sex'] = self.data['Sex'].map(sexo)
  # One Hot Encoding da variável Embarked
  embarked_dummies = pd.get_dummies(self.data['Embarked'])
  self.data = pd.concat([self.data, embarked_dummies], axis=1)

 def criar_variaveis(self) -> None:
  # Vamos somar a quantidade de irmãos e cônjuges,
  # a quantidade de pais e filhos,
  # e mais 1 para considera o próprio passageiro.
  self.data['FamilySize'] = self.data['SibSp'] + self.data['Parch'] + 1

 def remover_variaveis(self) -> None:
  "Remove as variáveis que não serão utilizadas pelo modelo para melhorar o desempenho"
  colunas_para_remover = [
  'PassengerId',
  'Name',
  'Ticket',
  'Cabin', # Variável com muitos dados faltantes.
  'Embarked', # Foram criadas variáveis dummies.
  'SibSp', 'Parch' # Foram combinadas em uma nova variável.
  ]
  self.data.drop(columns=colunas_para_remover, inplace=True)

 def normalizar_dados(self) -> None:
  variaveis = self.data.drop(columns='Survived')
  var_cols = variaveis.columns
  resposta = self.data['Survived']
  scaler = MinMaxScaler()
  variaveis = scaler.fit_transform(variaveis)
  variaveis = pd.DataFrame(variaveis, columns=var_cols)
 
  self.data = pd.concat([variaveis, resposta], axis=1)

 def separar_treino_teste(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
  "Separa a base de dados entre conjunto de treinamento e teste."
  treino, teste = train_test_split(self.data, test_size=0.3, random_state=2023)
  return treino, teste

 def preparar_dados(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
  "Executa todas as etapas de transformação de dados."
  self.tratar_nulos()
  self.tratar_variaveis_categoricas()
  self.criar_variaveis()
  self.remover_variaveis()
  self.normalizar_dados()
  treino, teste = self.separar_treino_teste()
  return treino, teste

In [ ]:
#Preparando os dados para a modelagem

df = pd.read_csv('/content/train.csv') #Caminho do arquivo
dp = DataPrep(df)
# Toda a preparação de dados necessaria e retorna os DataFrames de treino e teste já separados
df_treino, df_teste = dp.preparar_dados()

#Treinando o modelo com os conjuntos de dados.
#Separando as variáveis preditoras da variável resposta para 
#ambos DataFrames - treino e teste
X_treino = df_treino.drop(columns='Survived')
Y_treino = df_treino['Survived']

X_teste = df_teste.drop(columns='Survived')
Y_teste = df_teste['Survived']

#Instanciando o objeto
clf = LogisticRegression()

#Teinando o modelo utilizando os dados de treinamento.
clf.fit(X_treino, Y_treino)
#Avaliando a performance do modelo utilizando o conjunto de dados de teste 
#78% de acurácia com os dados tratados
print("Acurácia do modelo: ", clf.score(X_teste, Y_teste),"\n\n")


"""
Em projetos de machine learning é comum termos muitas possíveis variáveis preditoras 
ou criarmos variáveis que não têm um grande poder preditivo. Ambos os casos devem ser evitados. 
Ter muitas variáveis preditoras irá aumentar a complexidade do modelo, em muitos casos desnecessariamente, 
e pode fazer com que o algoritmo treinado tenha 
dificuldade de fazer previsões apuradas para novos dados. O mesmo pode acontecer quando temos variáveis com baixo poder preditivo.
"""

Acurácia do modelo:  0.7835820895522388 




<ipython-input-7-258844417806>:16: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.fillna(x.median()))


'\nEm projetos de machine learning é comum termos muitas possíveis variáveis preditoras \nou criarmos variáveis que não têm um grande poder preditivo. Ambos os casos devem ser evitados. \nTer muitas variáveis preditoras irá aumentar a complexidade do modelo, em muitos casos desnecessariamente, \ne pode fazer com que o algoritmo treinado tenha \ndificuldade de fazer previsões apuradas para novos dados. O mesmo pode acontecer quando temos variáveis com baixo poder preditivo.\n'